In [1]:
import pandas as pd
import numpy as np
import xgboost

In [2]:
df = pd.read_csv('../../all_data/xgboost_month_freq_dataset.csv')

# 補值(先隨便補)

In [3]:
df_clean = df.copy()

In [4]:
df_clean.info(max_cols=149)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806578 entries, 0 to 806577
Data columns (total 149 columns):
 #    Column                         Non-Null Count   Dtype  
---   ------                         --------------   -----  
 0    usage_category                 781214 non-null  object 
 1    address                        806578 non-null  object 
 2    property_type                  806566 non-null  object 
 3    community_name                 543379 non-null  object 
 4    main_building_ratio            750580 non-null  float64
 5    city                           806578 non-null  object 
 6    parking_space_price            190741 non-null  float64
 7    transaction_date               806578 non-null  object 
 8    elevator_available             288775 non-null  float64
 9    building_age                   611128 non-null  float64
 10   number_of_land                 806578 non-null  int64  
 11   number_of_building             806578 non-null  int64  
 12   number_of_park

In [5]:
# 處理坪數有0的，drop掉好了....
df_clean[ df_clean['total_area_ping'] == 0 ]

df_clean.drop(175950, axis=0, inplace=True)


In [6]:
df_clean['NA1005'].fillna(method='bfill', inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\1282215795.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['NA1005'].fillna(method='bfill', inplace=True)
C:\Users\student\AppData\Local\Temp\ipykernel_9544\1282215795.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean['NA1005'].fillna(method='bfill', inplace=True)


In [7]:
# transaction_floors target_floor total_floors detailed_usage_classification

In [8]:
df_clean['transaction_floors'].fillna(value=1, inplace=True)
df_clean['target_floor'].fillna(df_clean['target_floor'].mean(), inplace=True)
df_clean['total_floors'].fillna(df_clean['total_floors'].mean(), inplace=True)
df_clean['detailed_usage_classification'].fillna(value='其他', inplace=True)



C:\Users\student\AppData\Local\Temp\ipykernel_9544\3423363900.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['transaction_floors'].fillna(value=1, inplace=True)
C:\Users\student\AppData\Local\Temp\ipykernel_9544\3423363900.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [9]:
df_clean['total_floors'].mean()

12.000648550328503

In [10]:
df_clean['detailed_usage_classification'].isnull().sum()

0

In [11]:
# 要丟掉的欄位
df_clean.drop(['parking_space_price', 'elevator_available', 'note', 'year_month_for_combine', 'date', 'cor', 'latitude', 'longitude', 'address','price_per_ping'],axis=1, inplace=True)

In [12]:
df_clean.drop( 'transaction_date',axis=1, inplace=True)

In [13]:
# 經濟指標像後補值
df_clean.iloc[:,108:149].fillna(method='bfill', inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\2967026026.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_clean.iloc[:,108:149].fillna(method='bfill', inplace=True)


In [14]:
# building_age補平均值(其實應該要補當年的平均)
df_clean['building_age'].fillna(df_clean['building_age'].mean(), inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\3826513975.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['building_age'].fillna(df_clean['building_age'].mean(), inplace=True)


In [15]:
df_clean.isnull().sum()

usage_category          25364
property_type              12
community_name         263198
main_building_ratio     55997
city                        0
                        ...  
杜拜輕原油現貨價                    0
水泥50kg北區台泥                  0
無鉛汽油95                      0
竹節鋼筋_南                      0
預拌混凝土3000磅台北                0
Length: 138, dtype: int64

In [16]:
df_clean['property_type'].fillna(value="其他", inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\3971976473.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['property_type'].fillna(value="其他", inplace=True)


In [17]:
df_clean['main_building_ratio'].fillna(df_clean['main_building_ratio'].mean(), inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\2079150644.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['main_building_ratio'].fillna(df_clean['main_building_ratio'].mean(), inplace=True)


In [18]:
df_clean['community_name'].fillna(value="沒有社區", inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\3680213287.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['community_name'].fillna(value="沒有社區", inplace=True)


In [19]:
# 空值的話就先補"其他"
df_clean['usage_category'].unique()

array(['住', '商', '其他', '6', '工', '7', '農', nan], dtype=object)

In [20]:
# 把null值跟"6","7"，都改成"其他"
df_clean['usage_category'].fillna(value="其他", inplace=True)

for index, value in enumerate(df_clean['usage_category']):
    if value == "6":
        df_clean['usage_category'].iloc[index] = "其他"
    if value == "7":
        df_clean['usage_category'].iloc[index] = "其他"

C:\Users\student\AppData\Local\Temp\ipykernel_9544\3570540452.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['usage_category'].fillna(value="其他", inplace=True)
C:\Users\student\AppData\Local\Temp\ipykernel_9544\3570540452.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the orig

In [21]:
# 補平均
df_clean['bedrooms'].fillna(df_clean['bedrooms'].mean(), inplace=True)
df_clean['living_rooms'].fillna(df_clean['living_rooms'].mean(), inplace=True)
df_clean['bathrooms'].fillna(df_clean['bathrooms'].mean(), inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\2736197432.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['bedrooms'].fillna(df_clean['bedrooms'].mean(), inplace=True)
C:\Users\student\AppData\Local\Temp\ipykernel_9544\2736197432.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [22]:
# 以總價/總坪(不考慮是否有車位)，做為每坪的價格
# round(2): 取到小數點以下兩位
df_clean['new_per_ping'] = (df_clean['total_price'] / df_clean['total_area_ping']).round(2)

# 有缺補平均
df_clean['new_per_ping'].fillna(df_clean['new_per_ping'].mean(), inplace=True)

C:\Users\student\AppData\Local\Temp\ipykernel_9544\2119278460.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_clean['new_per_ping'].fillna(df_clean['new_per_ping'].mean(), inplace=True)


# 編碼

In [23]:
# 非數值變數
non_numeric_columns = [
    "usage_category",
    "property_type",
    "city",
    "per_ping_calculation_method",
    "detailed_usage_classification",
    "property_b_l_p_type",
    "district"
]
# one-hot encoding 

# df_clean['property_type'].unique()

In [24]:
# "community_name", # 這跟分0,1就好
# for index, value in enumerate(df_clean['community_name']):
#     if value == "沒有社區":
#         df_clean['community_name'].iloc[index] = 0
#     else:
#         df_clean['community_name'].iloc[index] = 1
# 這要20mins....

### 可能比較快的做法 #### 超快....0 second
# 使用 np.where 方法分配二進制標記
df_clean['community_name'] = np.where(df_clean['community_name'].isin(["沒有社區"]), 0, 1)


In [25]:
df_clean.drop('usage_category', axis=1, inplace=True)

In [26]:
# usage_category, detailed_usage_classification很類似，先留詳細的(detailed)

# df_clean = pd.get_dummies(df_clean, columns=['usage_category'], prefix='usage_category_')

df_clean = pd.get_dummies(df_clean, columns=['property_type'], prefix='property_type_')
df_clean = pd.get_dummies(df_clean, columns=['city'], prefix='city_')
df_clean = pd.get_dummies(df_clean, columns=['per_ping_calculation_method'], prefix='per_ping_calculation_method_')
df_clean = pd.get_dummies(df_clean, columns=['detailed_usage_classification'], prefix='detailed_usage_classification')
df_clean = pd.get_dummies(df_clean, columns=['property_b_l_p_type'], prefix='property_b_l_p_type_')
df_clean = pd.get_dummies(df_clean, columns=['district'], prefix='district_')

In [27]:
df_clean.isnull().sum().sum()

0

# XGBoost

In [28]:
import xgboost
from xgboost import XGBRFRegressor

In [29]:
df_clean.info(max_cols=205)

<class 'pandas.core.frame.DataFrame'>
Index: 806577 entries, 0 to 806577
Data columns (total 205 columns):
 #    Column                                                     Non-Null Count   Dtype  
---   ------                                                     --------------   -----  
 0    community_name                                             806577 non-null  int32  
 1    main_building_ratio                                        806577 non-null  float64
 2    building_age                                               806577 non-null  float64
 3    number_of_land                                             806577 non-null  int64  
 4    number_of_building                                         806577 non-null  int64  
 5    number_of_parking_space                                    806577 non-null  int64  
 6    management_org_available                                   806577 non-null  int64  
 7    total_area_ping                                            806577 non-null  f

In [30]:
X = df_clean.drop(['new_per_ping', 'total_price'], axis=1)
y = df_clean['new_per_ping']

In [31]:
# 切分資料集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=777, test_size=0.2)

In [32]:

dtrain = xgboost.DMatrix(X_train, label=y_train)
dtest = xgboost.DMatrix(X_test, label=y_test)

In [33]:
params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}
num_boost_round = 1000

In [80]:
# 簡言之這邊是用來測試"幾棵樹(weak learners)"比較好，用來調最後的num_boost_round超參數
# 每一棵樹做10 fold cross-validation 來衡量模型好壞(取10次平均)
# num_boost_round: "最多"總共可以長出幾棵樹(每一棵樹都是一樣weak learner，)
# early_stopping_rounds = n: 若近n次的衡量結果沒什麼變動，那就停止訓練(樹就不繼續長了)

cv_results = xgboost.cv(
    params,               # 其他超參數
    dtrain,               # 訓練資料集
    num_boost_round=10000, # 最多可以長到幾棵樹     # !!!看來還可以再深!!! # 2844
    seed=39,         # 隨機種子測試時要固定
    nfold=10,        # 每個模型跑幾次交叉驗證
    metrics={'rmse'}, # 評估的標準
    early_stopping_rounds=15  # 提早停止的標準
)
cv_results

c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:56:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:56:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:56:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

train-rmse-mean    1.609226e+06
train-rmse-std     3.708292e+05
test-rmse-mean     8.998483e+05
test-rmse-std      1.458828e+06
dtype: float64

In [48]:
# 看哪個index有最小的test-mae-mean
cv_results.idxmin()   # 2884

train-mae-mean    29857.110240
train-mae-std       129.824250
test-mae-mean     56295.936522
test-mae-std       6233.479288
Name: 2844, dtype: float64

In [35]:
# 調整這兩個超參 max_depth, min_child_weight

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(9,12)
    for min_child_weight in range(5,8)
] # 9 elements
gridsearch_params

[(9, 5), (9, 6), (9, 7), (10, 5), (10, 6), (10, 7), (11, 5), (11, 6), (11, 7)]

In [82]:
# Define initial best params and MAE   
# Best params: 10, 5, MAE: 64972.64892741657
min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgboost.cv(             # cv_results會是一個dataframe，會有選擇評估標準的column名稱
        params,                          # ex: metrics={'mae'}, 會有train-mae-mean,test-mae-mean
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'rmse'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-rmse-mean'].min()   # 改成算rmse
    boost_rounds = cv_results['test-rmse-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with max_depth=9, min_child_weight=5


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:58:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:58:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:58:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 1107248.3348472877 for 16 rounds
CV with max_depth=9, min_child_weight=6


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:59:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:59:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [14:59:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 1100975.9358304485 for 51 rounds
CV with max_depth=9, min_child_weight=7


...

	MAE 1106758.9308136601 for 17 rounds
CV with max_depth=11, min_child_weight=6


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:01:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:01:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:01:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 1099737.414637231 for 57 rounds
CV with max_depth=11, min_child_weight=7


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [15:02:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 1099513.121961423 for 60 rounds
Best params: 11, 7, MAE: 1099513.121961423


In [ ]:
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
] # 16 elements

In [ ]:
# subsample, colsample
# Best params: 0.9, 1.0, MAE: 65361.50899472935

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgboost.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

CV with subsample=1.0, colsample=1.0


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:40:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:40:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:40:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 65449.73520929491 for 680 rounds
CV with subsample=1.0, colsample=0.9


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:46:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:46:15] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [19:46:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 65962.33303309506 for 616 rounds
CV with subsample=1.0, colsample=0.8


...

	MAE 78051.7970408187 for 20 rounds
CV with subsample=0.7, colsample=0.8


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 79057.74671043413 for 21 rounds
CV with subsample=0.7, colsample=0.7


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [20:20:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

	MAE 79887.33413761416 for 11 rounds
Best params: 0.9, 1.0, MAE: 65361.50899472935


In [70]:
# eta
# Best params: 0.01, MAE: 70986.35099466259

%time
# This can take some time…
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    %time cv_results = xgboost.cv(params, dtrain, num_boost_round=num_boost_round, seed=42, nfold=5, metrics=['mae'], early_stopping_rounds=10)
    
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

CPU times: total: 0 ns
Wall time: 0 ns
CV with eta=0.3


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

CPU times: total: 1min 49s
Wall time: 10.3 s
CV with eta=0.2


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:30:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

CPU times: total: 1min 55s
Wall time: 10.7 s
CV with eta=0.1


...

CV with eta=0.01


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

CPU times: total: 2min 37s
Wall time: 13 s
CV with eta=0.005


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:31:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:l

CPU times: total: 3min 26s
Wall time: 15.8 s
Best params: None, MAE: inf


In [75]:
b_params = {
    # Parameters that we are going to tune.
    'max_depth':10,
    'min_child_weight': 5,
    'eta': 0.01,
    'subsample': 0.9,
    'colsample_bytree': 1,
    'reg_lambda': 0.1,
    
    # Other parameters
    # 樂於決定任務的評估指標(這邊是regression)
    'objective':'reg:linear',
}

In [78]:
# further best model
b_num_boost_round = 1000
best_model = xgboost.train(
    b_params,
    dtrain,
    num_boost_round=b_num_boost_round,
    evals=[(dtest, "Test")]
)

[0]	Test-rmse:254485.43887


c:\Users\student\anaconda3\envs\web_scraping\lib\site-packages\xgboost\core.py:160: UserWarning: [13:49:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[1]	Test-rmse:252902.32121
[2]	Test-rmse:251256.47020
[3]	Test-rmse:249742.87271
[4]	Test-rmse:248179.03948
[5]	Test-rmse:246798.36479
[6]	Test-rmse:245400.07101
[7]	Test-rmse:243885.99295
[8]	Test-rmse:242432.80675
[9]	Test-rmse:241026.28327
[10]	Test-rmse:239619.84358
[11]	Test-rmse:238308.79521
[12]	Test-rmse:236965.60644
[13]	Test-rmse:235639.03488
[14]	Test-rmse:234667.59799
[15]	Test-rmse:233435.67442
[16]	Test-rmse:232694.29929
[17]	Test-rmse:231580.19674
[18]	Test-rmse:230406.60351
[19]	Test-rmse:229226.08078
[20]	Test-rmse:228056.59653
[21]	Test-rmse:226946.51375
[22]	Test-rmse:225929.90167
[23]	Test-rmse:224908.79318
[24]	Test-rmse:224419.73588
[25]	Test-rmse:223343.71279
[26]	Test-rmse:223049.07358
[27]	Test-rmse:222055.56067
[28]	Test-rmse:221126.12680
[29]	Test-rmse:220059.26164
[30]	Test-rmse:219243.76875
[31]	Test-rmse:219135.03828
[32]	Test-rmse:218103.82373
[33]	Test-rmse:217223.39689
[34]	Test-rmse:216383.64860
[35]	Test-rmse:216703.27374
[36]	Test-rmse:216067.69298
[

In [79]:
# 1000棵樹效果好一些
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print(f"train mae:{mean_absolute_error(best_model.predict(dtrain), y_train)}")
print(f"train MSE:{mean_squared_error(best_model.predict(dtrain), y_train)}")
r2_score_linear_train = r2_score(y_train, best_model.predict(dtrain))
print('R2_score (train): ', r2_score_linear_train)
print('-'*50)

print(f"test mae:{mean_absolute_error(best_model.predict(dtest), y_test)}")
print(f"test MSE:{mean_squared_error(best_model.predict(dtest), y_test)}")
r2_score_linear_test = r2_score(y_test, best_model.predict(dtest))
print('R2_score (test): ', r2_score_linear_test)


train mae:64526.78205300291
train MSE:243092060505.61533
R2_score (train):  0.9178082863187346
--------------------------------------------------
test mae:68427.60415815948
test MSE:73920792825.77776
R2_score (test):  -0.12698191996858244


In [52]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

best_model.fit(X_train, y_train)
y_test_pred_xg = best_model.predict(X_test)



# R2 score，即決定係數，反映Y的全部變異能通過迴歸關係被所有X解釋的比例。

# y_pred_linear_train = model_xg.fit(X_test) # test
r2_score_linear_test = r2_score(y_test, y_test_pred_xg)

# train
y_train_predict = best_model.predict(X_train) 
r2_score_linear_train = r2_score(y_train, y_train_predict)

# y_pred_linear_test = model_xg.predict(X_test)
# r2_score_linear_test = r2_score(y_test, y_pred_linear_test)

# RMSE一般指均方根誤差(標準誤差)，可衡量預測值和實際值之間的平均差異，藉此估計預測模型預測目標值的準確度(越低越好)
rmse_train = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
rmse_test = (np.sqrt(mean_squared_error(y_test, y_test_pred_xg)))

print('R2_score (train): ', r2_score_linear_train)
print('R2_score (test): ', r2_score_linear_test)
print("RMSE: ", rmse_train)
print("RMSE: ", rmse_test)

AttributeError: 'Booster' object has no attribute 'fit'

# 基礎形式的xgboost

In [ ]:
# model_xg = XGBRFRegressor()
# model_xg.fit(X_train, y_train)
# y_test_pred_xg = model_xg.predict(X_test)
# # metrics.accuracy_score(y_test, y_test_pred_xg)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# R2 score，即決定係數，反映Y的全部變異能通過迴歸關係被所有X解釋的比例。

# y_pred_linear_train = model_xg.fit(X_test) # test
r2_score_linear_test = r2_score(y_test, y_test_pred_xg)

# train
y_train_predict = model_xg.predict(X_train) 
r2_score_linear_train = r2_score(y_train, y_train_predict)

# y_pred_linear_test = model_xg.predict(X_test)
# r2_score_linear_test = r2_score(y_test, y_pred_linear_test)

# RMSE一般指均方根誤差(標準誤差)，可衡量預測值和實際值之間的平均差異，藉此估計預測模型預測目標值的準確度(越低越好)
rmse_train = (np.sqrt(mean_squared_error(y_train, y_train_predict)))
rmse_test = (np.sqrt(mean_squared_error(y_test, y_test_pred_xg)))

print('R2_score (train): ', r2_score_linear_train)
print('R2_score (test): ', r2_score_linear_test)
print("RMSE: ", rmse_train)
print("RMSE: ", rmse_test)

R2_score (train):  0.9344782935499697
R2_score (test):  0.38124705414878035
RMSE:  440214.1209514309
RMSE:  201457.51543300366
